# 数据集创建&处理

In [1]:
from dataset.build_dataset import build_dataset
import mindspore.dataset as ds
import os
import numpy as np
from mindspore.mindrecord import FileWriter
import json

In [2]:
# 创建mindrecord

# FEATURES_FILE = "./data/json_features.npy"
# features = np.load(FEATURES_FILE)
# list_dict = []
# for item in features:
#     dict_temp = json.loads(item)
#     list_dict.append(dict_temp)
# SQUAD_MINDRECORD_FILE = "./data/squad_features.mindrecord"

# if os.path.exists(SQUAD_MINDRECORD_FILE):
#     os.remove(SQUAD_MINDRECORD_FILE)
#     os.remove(SQUAD_MINDRECORD_FILE + ".db")

# writer = FileWriter(file_name=SQUAD_MINDRECORD_FILE, shard_num=1)

# data_schema = {
#     "word_ids": {"type": "int32", "shape": [-1]},
#     "word_segment_ids": {"type": "int32", "shape": [-1]},
#     "word_attention_mask": {"type": "int32", "shape": [-1]},
#     "entity_ids": {"type": "int32", "shape": [-1]},
#     "entity_position_ids": {"type": "int32", "shape": [-1]},
#     "entity_segment_ids": {"type": "int32", "shape": [-1]},
#     "entity_attention_mask": {"type": "int32", "shape": [-1]},
#     "start_positions": {"type": "int32", "shape": [-1]},
#     "end_positions": {"type": "int32", "shape": [-1]}
# }
# writer.add_schema(data_schema, "it is a preprocessed squad dataset")

# data = []
# i = 0
# for item in list_dict:
#     i += 1
#     sample = {
#         "word_ids": np.array(item["word_ids"], dtype=np.int32),
#         "word_segment_ids": np.array(item["word_segment_ids"], dtype=np.int32),
#         "word_attention_mask": np.array(item["word_attention_mask"], dtype=np.int32),
#         "entity_ids": np.array(item["entity_ids"], dtype=np.int32),
#         "entity_position_ids": np.array(item["entity_position_ids"], dtype=np.int32),
#         "entity_segment_ids": np.array(item["entity_segment_ids"], dtype=np.int32),
#         "entity_attention_mask": np.array(item["entity_attention_mask"], dtype=np.int32),
#         "start_positions": np.array(item["start_positions"], dtype=np.int32),
#         "end_positions": np.array(item["end_positions"], dtype=np.int32),
#     }

#     data.append(sample)
#     #print(sample)
#     if i % 10 == 0:
#         writer.write_raw_data(data)
#         data = []

# if data:
#     writer.write_raw_data(data)

# writer.commit()

In [2]:
SQUAD_MINDRECORD_FILE = "./data/squad_features.mindrecord"
data_set = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE)
count = 0
for item in data_set.create_dict_iterator():
    #print(item)
    count += 1
print("Got {} samples".format(count))

Got 269 samples


In [3]:
data_sample = next(data_set.create_dict_iterator())
data_sample

{'end_positions': Tensor(shape=[1], dtype=Int32, value= [101]),
 'entity_attention_mask': Tensor(shape=[2], dtype=Int32, value= [0, 0]),
 'entity_ids': Tensor(shape=[2], dtype=Int32, value= [0, 0]),
 'entity_position_ids': Tensor(shape=[60], dtype=Int32, value= [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 
  -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 
  -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]),
 'entity_segment_ids': Tensor(shape=[2], dtype=Int32, value= [0, 0]),
 'start_positions': Tensor(shape=[1], dtype=Int32, value= [101]),
 'word_attention_mask': Tensor(shape=[422], dtype=Int32, value= [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

# Model

In [4]:
from readingcomprehension.models.luke import LukeForReadingComprehension, LukeEntityAwareAttentionModel
import mindspore.common.dtype as mstype
from model.bert_model import BertConfig
from mindspore import context
from model.luke import LukeModel, EntityAwareEncoder
import numpy as np
from mindspore import Tensor, context
from mindspore import dtype as mstype
import mindspore.ops as ops
import mindspore.nn as nn
from model.bert_model import BertOutput
from mindspore.common.initializer import TruncatedNormal
import math
from mindspore.ops import composite as C
import mindspore
from mindspore.ops import operations as P
context.set_context(mode=context.GRAPH_MODE, device_target="CPU")

In [7]:
config = BertConfig()
Luke_model = LukeForReadingComprehension(config)

In [8]:
op_stack = ops.Stack()
word_ids = op_stack([data_sample["word_ids"], data_sample["word_ids"]])
word_segment_ids = op_stack([data_sample["word_segment_ids"], data_sample["word_segment_ids"]])
word_attention_mask = op_stack([data_sample["word_attention_mask"], data_sample["word_attention_mask"]])
entity_ids = op_stack([data_sample["entity_ids"], data_sample["entity_ids"]])
entity_position_ids = op_stack([data_sample["entity_position_ids"], data_sample["entity_position_ids"]])
entity_segment_ids = op_stack([data_sample["entity_segment_ids"], data_sample["entity_segment_ids"]])
entity_attention_mask = op_stack([data_sample["entity_attention_mask"], data_sample["entity_attention_mask"]])
start_positions = op_stack([data_sample["start_positions"], data_sample["start_positions"]])
end_positions = op_stack([data_sample["end_positions"], data_sample["end_positions"]])

In [9]:
model = LukeForReadingComprehension(config)
model.construct(word_ids,
                word_segment_ids,
                word_attention_mask,
                entity_ids,
                entity_position_ids,
                entity_segment_ids,
                entity_attention_mask,
                start_positions,
                end_positions
                )

(Tensor(shape=[2], dtype=Float32, value= [6.04547405e+000, 6.04547405e+000]),
 Tensor(shape=[2, 422], dtype=Float32, value=
 [[4.01165873e-001, 3.96056533e-001, 4.01458025e-001 ... 4.10320163e-001, 4.11856383e-001, 3.84537429e-001],
  [4.01165873e-001, 3.96056533e-001, 4.01458025e-001 ... 4.10320163e-001, 4.11856383e-001, 3.84537429e-001]]),
 Tensor(shape=[2, 422], dtype=Float32, value=
 [[4.01165873e-001, 3.96056533e-001, 4.01458025e-001 ... 4.10320163e-001, 4.11856383e-001, 3.84537429e-001],
  [4.01165873e-001, 3.96056533e-001, 4.01458025e-001 ... 4.10320163e-001, 4.11856383e-001, 3.84537429e-001]]))